<a href="https://colab.research.google.com/github/daiveeknaik14/BookRecommendationSystem/blob/master/BookRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Book Recomendation System based on **Collaborative Filtering using SVD**

---


## We will be using surprise for the implementation.
## Surprise is a scikit for recommendor systems. It is highly intuitive.
## Here we first install surprise using pip.

In [29]:
!pip3 install surprise

## Importing libraries that contain modules to be used throughout the implementatoion.

In [30]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [31]:
#inputing the data to dataframes from csv files
df_books = pd.read_csv('/content/drive/My Drive/Datasets/bookData/books.csv')
df_ratings = pd.read_csv('/content/drive/My Drive/Datasets/bookData/ratings.csv')

## Displyed the headers of both, the books and the ratings dataset.
## Note that here we could have used print() but it cuts out the columns in between and hence we use display the column header.

In [32]:
#displaying headers of both dataframes
display(df_books.head())
display(df_ratings.head())

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


## Given below is the breakdown of all the modules used in the code snipped used below


*   **Reader** : The Reader class is used to parse the file containing ratings.
*   **Dataset.load_from_df** : It is used to load dataset from pandas from dataframe. The parameters are 

            df (Dataframe) – The dataframe containing the ratings. It must have three columns, corresponding to the user (raw) ids, the item (raw) ids, and the ratings, in this order.

            reader (Reader) – A reader to read the file. Only the rating_scale field needs to be specified.



*   **SVD()** : We use this to initialise our algorithm to SVD.
*   **cross_validate** : We use cross_validate to calculate the RMSE and MAE for the validation dataset. RMSE and MAE are useful to assess the performance of the model.

# **NOTE:**

It is important to note here that in cross_validate we use KFolds method to evaluate our model. What is does is basically it splits the validation data set into different models and evaluate them seperately as unique groups. This removes the optimistic and biased analysis of other evaluation models.










# Why SVM ?

The whole concept of SVM is actually interesting. It basically is the PCA or Principal Component Analysis of, in our case, first the user preference of books which basically reduces the dimensionality to the specific genres.

 Then in the second calculation PCA is performed on the books selected by users which dimensionalise the books into genres. The multiplication of one matrix with the transpose of the other results in a matrix called R. The matrix R is then use to predict the values. This whole process is also called matrix factorisation.

The whole concept is brilliantly explained on the following link -

https://www.youtube.com/watch?v=8wLKuscyO9I&t=334s

In [12]:
#initialising, fitting, training and validating for our data
r = Reader(rating_scale=(1,5))
df = pd.merge(df_books, df_ratings, on='book_id')
data = Dataset.load_from_df(df_ratings[['book_id', 'user_id', 'rating']], r)
train, test = train_test_split(data, test_size=0.20)
model = SVD()
model.fit(train)
pred = model.test(test)
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8431  0.8442  0.8437  0.8432  0.8425  0.8433  0.0006  
MAE (testset)     0.6588  0.6604  0.6596  0.6594  0.6590  0.6595  0.0006  
Fit time          56.22   56.26   56.71   56.31   56.56   56.41   0.19    
Test time         2.27    2.22    2.32    2.26    2.21    2.25    0.04    


{'fit_time': (56.21846294403076,
  56.25603938102722,
  56.71034598350525,
  56.30690121650696,
  56.55851721763611),
 'test_mae': array([0.6588025 , 0.66040377, 0.65964248, 0.65943234, 0.65900693]),
 'test_rmse': array([0.84311969, 0.84418993, 0.84365167, 0.84320959, 0.84253342]),
 'test_time': (2.265202045440674,
  2.22481369972229,
  2.3191134929656982,
  2.255211353302002,
  2.209268569946289)}

## The following is the function that returns top 10 recommended books for the user depending on the user_id. It removes books that the user has already read.

In [27]:
def recommendor_function(user_id):
  #copying the books data to a variable
  user = df_books.copy()
  #finding out the books that the user has already read
  already_read = df[df['user_id'] == user_id]['book_id'].unique()
  #resetting indexes and removing read books from the user dataframe
  user = user.reset_index()
  user = user[~user['book_id'].isin(already_read)]
  #adding the estimated score to the last column
  user['Estimate_Score']=user['book_id'].apply(lambda x: model.predict(user_id, x).est)
  ser = user.drop('book_id', axis = 1)
  #displaying the top 10 recommended books in descending order
  user = user.sort_values('Estimate_Score', ascending=False)
  display(user.head(10))  

## Taking in the user id from the user.

In [24]:
user_id_for_rec = int(input("Enter your user id for recommendation: "))

Enter your user id for recommendation: 34


## The following call will display the top 10 recommended books for the user id that we entered.


---


# **Note that the Estimate Score for how much the user would like the book is displayed in the last column.**



In [28]:
recommendor_function(user_id_for_rec)

,index,id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,Estimate_Score
3127,3127,3128,43782,43191,57,009947137X,9.780099e+12,Anne Rice,1989.0,The Mummy,The Mummy (Ramses the Damned #1),eng,3.81,34795,38317,850,812,3206,10236,12090,11973,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,4.180672
8994,8994,8995,8087,2321293,10,441012396,9.780441e+12,Robin McKinley,1993.0,Deerskin,Deerskin,en-US,3.88,15982,17728,1124,620,1328,3806,5810,6164,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,4.087901
1410,1410,1411,15622,3159084,103,006083756X,9.780061e+12,Richard Wright,1940.0,Native Son,Native Son,eng,3.96,66194,70034,2918,1828,3920,14032,25876,24378,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,4.040164
2670,2670,2671,41977,3152550,82,99498596,9.780099e+12,Maeve Binchy,1990.0,Circle of Friends,Circle of Friends,eng,4.00,43936,47875,1469,381,1670,10917,19654,15253,https://images.gr-assets.com/books/1342486126m...,https://images.gr-assets.com/books/1342486126s...,3.967363
1985,1985,1986,5345,3202981,101,385517238,9.780386e+12,John Grisham,2006.0,The Innocent Man,The Innocent Man: Murder and Injustice in a Sm...,eng,3.79,43793,48240,3891,1412,3719,12250,17282,13577,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,3.963375
6664,6664,6665,31335,2925856,35,345396936,9.780345e+12,Anne Rice,1982.0,Cry to Heaven,Cry to Heaven,en-US,3.83,16988,17794,601,467,1486,4449,5576,5816,https://images.gr-assets.com/books/1385216547m...,https://images.gr-assets.com/books/1385216547s...,3.955236
6062,6062,6063,43037,320876,21,192835149,9.780193e+12,Jack London,1906.0,"The Call of the Wild, White Fang and Other Sto...","The Call of the Wild, White Fang and Other Sto...",eng,3.97,16717,17318,234,316,868,3894,6215,6025,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,3.919712
711,711,712,10988,3123704,96,385335989,9.780385e+12,Diana Gabaldon,1996.0,"Drums of Autumn (Outlander, #4)","Drums of Autumn (Outlander, #4)",eng,4.34,123736,145728,5743,584,2679,18529,48756,75180,https://images.gr-assets.com/books/1500688941m...,https://images.gr-assets.com/books/1500688941s...,3.903955
9792,9792,9793,15677,1187279,77,393321983,9.780393e+12,Patricia Highsmith,1950.0,Strangers on a Train,Strangers on a Train,eng,3.82,9788,11500,1055,195,720,2843,4922,2820,https://images.gr-assets.com/books/1331234879m...,https://images.gr-assets.com/books/1331234879s...,3.844877
2740,2740,2741,6690,824600,125,375814256,9.780376e+12,"Roald Dahl, Quentin Blake",1975.0,Danny: The Champion of the World,Danny the Champion of the World,NaN,4.07,39802,45049,1844,496,1933,9393,15259,17968,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,3.824747
